In [5]:
# Imports

import os
import json
import tqdm
import yaml

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from omegaconf import OmegaConf
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
%%bash
mkdir -p results
fusermount -uz results
sshfs compute.ies:/mnt/work/dhuseljic/results/uncertainty results
ls results

fusermount: entry for /home/denis/Documents/projects/uncertainty_evaluation/experiments/uncertainty/notebooks/results not found in /etc/mtab


baselines
CIFAR10


In [9]:
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def load_results(path):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
        
    results = {}

    exp_json = path / 'results_final.json'
    exp_cfg = path / '.hydra' / 'config.yaml'
    try:
        cfg =  OmegaConf.load(exp_cfg)
        data = load_json(exp_json)
    except:
        print(f'{path} has missing results.')
        return

    results['cfg'] = cfg
    # results['checkpoint'] = checkpoint
    results['results'] = data
    return results

def get_experiments(result_path, glob_pattern, train_results=False):
    # Aggregate results over multiple glob pattern such as seeds
    experiments = []
    for exp_path in result_path.glob(glob_pattern):
        d = load_results(exp_path)
        experiments.append(d)
    assert len(experiments) != 0, f'No experiments found for {result_path}.'
    return experiments

path = f'results/baselines/CIFAR10/resnet18/'
exp_results = get_experiments(Path(path), 'seed1')
# metrics = get_metric_values(exp_results)
print(exp_results[0]['results'].keys())
# print(len(exp_results[0]['results']['train_history']))
# print(exp_results[0]['results']['test_stats'].keys())

dict_keys(['test_stats', 'misc'])


In [24]:
# Baseline results
dataset = 'CIFAR10'
experiments = {}
experiments.update({
    'standard': f'results/baselines/CIFAR10/resnet18/',
    'label_smoothing': f'results/baselines/CIFAR10/resnet18_labelsmoothing/',
    # 'mixup': f'results/baselines/CIFAR10/resnet18_mixup/',
    'sngp': f'results/baselines/CIFAR10/resnet18_sngp/',
})


all_results = {}
for exp_name, exp_path in tqdm(experiments.items()):
    all_results[exp_name] = get_experiments(Path(exp_path), 'seed1')


100%|██████████| 3/3 [00:00<00:00, 134.93it/s]


In [25]:
result_dict = {}

for exp_name in experiments:
    d = {}
    for metric_key in ['accuracy', 'brier', 'ece', 'ace', 'auroc_SVHN', 'auroc_CIFAR100']: 
        vals = [seed_results['results']['test_stats'][metric_key] for seed_results in all_results[exp_name]]
        d[metric_key] = np.mean(vals)
    result_dict[exp_name] = d
print(pd.DataFrame(result_dict).T.to_markdown())

|                 |   accuracy |     brier |       ece |       ace |   auroc_SVHN |   auroc_CIFAR100 |
|:----------------|-----------:|----------:|----------:|----------:|-------------:|-----------------:|
| standard        |     0.9489 | 0.082059  | 0.0274236 | 0.036072  |     0.885258 |         0.874181 |
| label_smoothing |     0.9503 | 0.0793098 | 0.0402002 | 0.0340034 |     0.817936 |         0.814172 |
| sngp            |     0.8614 | 0.200464  | 0.0214757 | 0.0247449 |     0.882446 |         0.822672 |
